## 算法说明

Xception是对Inception v3的另一种改进，采用depthwise separable convolution来替换原来Inception v3中的卷积操作。  
下面对depthwise separable convolution进行简单介绍： 
  
depthwise separable convolution包含两个过程：depthwise convolution 和 pointwise convolution：
- depthwise convolution对输入的每一个channel独立的用对应channel的所有卷积核去卷积。假设卷积核的shape是[ filter_height, filter_width, in_channels, channel_multiplier ]，那么每个in_channel会输出channel_multiplier那么多个通道，最后的feature map就会有in_channels * channel_multiplier个通道了。
- pointwise convolution是 1x1 卷积操作，是对多个分散通道进行融合的操作。  
具体操作如下图所示：  
![pic0](pic0.png)  
   
Xception是基于depthwise separable convolution构建的分类架构，其网络架构如图所示：  
![pic1](pic1.png)  
基于上述架构图，对Xception进行编码实现。

## 算法实现

In [16]:
import keras

def middle_flow(x, block, name=None):
    '''
    middle flow block
    :param x: input tensor
    :param block: index of block for name
    :param name: operation name
    :return: output tensor
    '''
    shortcut = x
    x = keras.layers.Activation('relu', name=name+str(block)+'_relu0')(x)
    x = keras.layers.SeparableConv2D(filters=728, kernel_size=(3,3), padding='same', name=name+'_res'+str(block)+'_SepConv0')(x)
    x = keras.layers.BatchNormalization(name=name+str(block)+'_bn0')(x)
    x = keras.layers.Activation('relu', name=name+str(block)+'_relu1')(x)
    x = keras.layers.SeparableConv2D(filters=728, kernel_size=(3,3), padding='same', name=name+str(block)+'_SepConv1')(x)
    x = keras.layers.BatchNormalization(name=name+str(block)+'_bn1')(x)
    x = keras.layers.Activation('relu', name=name+str(block)+'_relu2')(x)
    x = keras.layers.SeparableConv2D(filters=728, kernel_size=(3,3), padding='same', name=name+str(block)+'_SepConv2')(x)
    x = keras.layers.BatchNormalization(name=name+str(block)+'_bn2')(x)
    x = keras.layers.Add(name=name+str(block)+'_add')([shortcut, x])
    return x

def Xception(input_shape=(299,299,3), classes = 1000, use_dropout = True, dropout_rate = 0.2):
    
    # Entry flow 1
    x_input = keras.layers.Input(shape=input_shape, name='input_layer')
    x = keras.layers.Conv2D(filters=32, kernel_size=(3,3), strides=2, padding='same', name='entryflow_conv0')(x_input)
    x = keras.layers.BatchNormalization(name='entryflow_bn0')(x)
    x = keras.layers.Activation('relu', name='entryflow_relu0')(x)  
    x = keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding='same', name='entryflow_conv1')(x)
    x = keras.layers.BatchNormalization(name='entryflow_bn1')(x)
    x = keras.layers.Activation('relu', name='entryflow_relu1')(x)
    
    # Entry flow 2
    shortcut = keras.layers.Conv2D(filters=128, kernel_size=(1,1), strides=2, padding='valid', name='entryflow_res0_shortcut')(x)
    shortcut = keras.layers.BatchNormalization(name='entryflow_res0_shortcutbn')(shortcut) 
    x = keras.layers.SeparableConv2D(filters=128, kernel_size=(3,3), padding='same', name='entryflow_res0_SepConv0')(x)
    x = keras.layers.BatchNormalization(name='entryflow_res0_bn0')(x)
    x = keras.layers.Activation('relu', name='entryflow_res0_relu0')(x)
    x = keras.layers.SeparableConv2D(filters=128, kernel_size=(3,3),padding='same', name='entryflow_res0_SepConv1')(x)
    x = keras.layers.BatchNormalization(name='entryflow_res0_bn1')(x)
    x = keras.layers.MaxPooling2D(pool_size=(3,3), strides=2, padding='same', name='entryflow_res0_pool')(x)
    x = keras.layers.Add(name='entryflow_res0_add')([shortcut, x])
    
    #Entry flow 3
    shortcut = keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=2, padding='same', name='entryflow_res1_shortcut')(x)
    shortcut = keras.layers.BatchNormalization(name='entryflow_res1_shortcutbn')(shortcut)
    x = keras.layers.Activation('relu', name='entryflow_res1_relu0')(x)
    x = keras.layers.SeparableConv2D(filters=256, kernel_size=(3,3), padding='same', name='entryflow_res1_SepConv0')(x)
    x = keras.layers.BatchNormalization(name='entryflow_res1_bn0')(x)
    x = keras.layers.Activation('relu', name='entryflow_res1_relu1')(x)
    x = keras.layers.SeparableConv2D(filters=256, kernel_size=(3,3), padding='same', name='entryflow_res1_SepConv1')(x)
    x = keras.layers.BatchNormalization(name='entryflow_res1_bn1')(x)
    x = keras.layers.MaxPooling2D(pool_size=(3,3), strides=2, padding='same', name='entryflow_res1_pool')(x)  
    x = keras.layers.Add(name='entryflow_res1_add')([shortcut, x])
    
    #Entry flow 4
    shortcut = keras.layers.Conv2D(filters=728, kernel_size=(1,1), strides=2, padding='same', name='entryflow_res2_shortcut')(x)
    shortcut = keras.layers.BatchNormalization(name='entryflow_res2_shortcutbn')(shortcut)
    x = keras.layers.Activation('relu', name='entryflow_res2_relu0')(x)
    x = keras.layers.SeparableConv2D(filters=728, kernel_size=(3,3), padding='same', name='entryflow_res2_SepConv0')(x)
    x = keras.layers.BatchNormalization(name='entryflow_res2_bn0')(x)
    x = keras.layers.Activation('relu', name='entryflow_res2_relu1')(x)
    x = keras.layers.SeparableConv2D(filters=728, kernel_size=(3,3), padding='same', name='entryflow_res2_SepConv1')(x)
    x = keras.layers.BatchNormalization(name='entryflow_res2_bn1')(x)
    x = keras.layers.MaxPooling2D(pool_size=(3,3), strides=2, padding='same', name='entryflow_res2_pool')(x)
    x = keras.layers.Add(name='entryflow_res2_add')([shortcut, x])
    
    #Middle flow
    #重复8次
    for i in range(8):
        x = middle_flow(x, i+3, name='middleflow')
    
    #Exit flow
    shortcut = keras.layers.Conv2D(filters=1024, kernel_size=(1,1), strides=2, padding='same', name='exitflow_shortcut')(x)
    shortcut = keras.layers.BatchNormalization(name='exitflow_shortcutbn')(shortcut)
    x = keras.layers.Activation('relu', name='exitflow_relu0')(x)
    x = keras.layers.SeparableConv2D(filters=728, kernel_size=(3,3), padding='same', name='exitflow_SepConv0')(x)
    x = keras.layers.BatchNormalization(name='exitflow_bn0')(x)
    x = keras.layers.Activation('relu', name='exitflow_relu1')(x)
    x = keras.layers.SeparableConv2D(filters=1024, kernel_size=(3,3), padding='same', name='exitflow_SepConv1')(x)
    x = keras.layers.BatchNormalization(name='exitflow_bn1')(x)
    x = keras.layers.MaxPooling2D(pool_size=(3,3), strides=2, padding='same', name='exitflow_pool')(x)   
    x = keras.layers.Add(name='exitflow_add')([shortcut, x])
    
    x = keras.layers.SeparableConv2D(filters=1536, kernel_size=(3,3), padding='same', name='exitflow_SepConv2')(x)
    x = keras.layers.BatchNormalization(name='exitflow_bn2')(x)
    x = keras.layers.Activation('relu', name='exitflow_relu2')(x)
    x = keras.layers.SeparableConv2D(filters=2048, kernel_size=(3,3), padding='same', name='exitflow_SepConv3')(x)
    x = keras.layers.BatchNormalization(name='exitflow_bn3')(x)
    x = keras.layers.Activation('relu', name='exitflow_relu3')(x)
    
    x = keras.layers.GlobalAveragePooling2D(name='exitflow_globalaveragepool')(x)
    
    #dense layer
    #dropout
    if use_dropout:
        x = keras.layers.Dropout(dropout_rate)(x)
        
    #dense-layer
    x = keras.layers.Dense(units = classes, activation='softmax', kernel_initializer='glorot_uniform', name = 'fc' + str(classes))(x)
    model = keras.models.Model(inputs = x_input, outputs = x, name = 'Xception')
    return model

In [17]:
model = Xception()
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
entryflow_conv0 (Conv2D)        (None, 150, 150, 32) 896         input_layer[0][0]                
__________________________________________________________________________________________________
entryflow_bn0 (BatchNormalizati (None, 150, 150, 32) 128         entryflow_conv0[0][0]            
__________________________________________________________________________________________________
entryflow_relu0 (Activation)    (None, 150, 150, 32) 0           entryflow_bn0[0][0]              
__________________________________________________________________________________________________
entryflow_

## 参考文献
[1]Chollet F. Xception: Deep learning with depthwise separable convolutions[J]. arXiv preprint, 2017: 1610.02357.  